In [3]:
import pandas as pd
import os
pd.options.display.max_columns = None
pd.options.display.max_rows = None
os.chdir('/root/netscope-main/')

from analysis.eval_AD import *
from analysis.TSE import *
from analysis.sketch_based_AD import *

In [4]:
def get_flows_df(base_dir):
    raw_flow_latency_dict, raw_flow_qunatile_dict,raw_flow_c_plus_dict, raw_flow_c_minus_dict, \
    raw_flow_count_dict, raw_flow_perc_dict,raw_flow_max_gap_dict, raw_flow_min_gap_dict, \
    raw_flow_lambda_dict,raw_flow_prev_max_gap_dict,raw_flow_max_value_dict, raw_flow_min_value_dict,\
        raw_flow_recv_ts_dict,raw_flow_MARS_AD_dict, raw_flow_src_port_dict, raw_flow_dst_port_dict = read_from_int_file(base_dir)

    raw_flow_latency_df =  pd.DataFrame(pd.DataFrame.from_dict(raw_flow_latency_dict, orient='index').values.T, columns=list(raw_flow_latency_dict.keys()))
    raw_flow_qunatile_df =  pd.DataFrame(pd.DataFrame.from_dict(raw_flow_qunatile_dict, orient='index').values.T, columns=list(raw_flow_qunatile_dict.keys()))

    raw_flow_c_plus_df =  pd.DataFrame(pd.DataFrame.from_dict(raw_flow_c_plus_dict, orient='index').values.T, columns=list(raw_flow_c_plus_dict.keys()))
    raw_flow_c_minus_df =  pd.DataFrame(pd.DataFrame.from_dict(raw_flow_c_minus_dict, orient='index').values.T, columns=list(raw_flow_c_minus_dict.keys()))
    raw_flow_count_df =  pd.DataFrame(pd.DataFrame.from_dict(raw_flow_count_dict, orient='index').values.T, columns=list(raw_flow_count_dict.keys()))
    raw_flow_perc_df =  pd.DataFrame(pd.DataFrame.from_dict(raw_flow_perc_dict, orient='index').values.T, columns=list(raw_flow_perc_dict.keys()))

    raw_flow_max_gap_df =  pd.DataFrame(pd.DataFrame.from_dict(raw_flow_max_gap_dict, orient='index').values.T, columns=list(raw_flow_max_gap_dict.keys()))
    raw_flow_min_gap_df =  pd.DataFrame(pd.DataFrame.from_dict(raw_flow_min_gap_dict, orient='index').values.T, columns=list(raw_flow_min_gap_dict.keys()))
    raw_flow_lambda_df =  pd.DataFrame(pd.DataFrame.from_dict(raw_flow_lambda_dict, orient='index').values.T, columns=list(raw_flow_lambda_dict.keys()))
    raw_flow_prev_max_gap_df =  pd.DataFrame(pd.DataFrame.from_dict(raw_flow_prev_max_gap_dict, orient='index').values.T, columns=list(raw_flow_prev_max_gap_dict.keys()))

    raw_flow_max_value_df =  pd.DataFrame(pd.DataFrame.from_dict(raw_flow_max_value_dict, orient='index').values.T, columns=list(raw_flow_max_value_dict.keys()))
    raw_flow_min_value_df =  pd.DataFrame(pd.DataFrame.from_dict(raw_flow_min_value_dict, orient='index').values.T, columns=list(raw_flow_min_value_dict.keys()))

    raw_flow_recv_ts_df =  pd.DataFrame(pd.DataFrame.from_dict(raw_flow_recv_ts_dict, orient='index').values.T, columns=list(raw_flow_recv_ts_dict.keys()))

    raw_flow_MARS_AD_df =  pd.DataFrame(pd.DataFrame.from_dict(raw_flow_MARS_AD_dict, orient='index').values.T, columns=list(raw_flow_MARS_AD_dict.keys()))

    raw_flow_src_port_df =  pd.DataFrame(pd.DataFrame.from_dict(raw_flow_src_port_dict, orient='index').values.T, columns=list(raw_flow_src_port_dict.keys()))
    raw_flow_dst_port_df =  pd.DataFrame(pd.DataFrame.from_dict(raw_flow_dst_port_dict, orient='index').values.T, columns=list(raw_flow_dst_port_dict.keys()))


    merge_flow_info_dict = merge_flow_info(raw_flow_latency_df, raw_flow_max_value_df, raw_flow_min_value_df, raw_flow_qunatile_df, raw_flow_recv_ts_df,raw_flow_MARS_AD_df, raw_flow_src_port_df, raw_flow_dst_port_df)
    return merge_flow_info_dict
def calc_threshold(merge_flow_info_dict, max_ratio = 1, sigma = 3):
    for flow in merge_flow_info_dict.keys():
        merge_flow_info_dict[flow]['sigma'] = (merge_flow_info_dict[flow]['max']/max_ratio + merge_flow_info_dict[flow]['min']) / 4
        merge_flow_info_dict[flow]['threshold'] = merge_flow_info_dict[flow]['quantile'] + sigma * merge_flow_info_dict[flow]['sigma']
    return merge_flow_info_dict

def anomaly_detection_static_threshold(merge_flow_info_dict):
    AD_T_100 = []
    AD_T_500 = []
    AD_T_1000 = []
    AD_T_5000 = []
    array_length = 0
    for flow in merge_flow_info_dict.keys():
        array_length = len(merge_flow_info_dict[flow]["latency"])
        merge_flow_info_dict[flow]["100_T_AD"] = merge_flow_info_dict[flow]['latency'] > 100000
        merge_flow_info_dict[flow]["500_T_AD"] = merge_flow_info_dict[flow]['latency'] > 500000
        merge_flow_info_dict[flow]["1000_T_AD"] = merge_flow_info_dict[flow]['latency'] > 1000000
        merge_flow_info_dict[flow]["5000_T_AD"] = merge_flow_info_dict[flow]['latency'] > 5000000
        AD_T_100 = AD_T_100 + merge_flow_info_dict[flow].index[merge_flow_info_dict[flow]["100_T_AD"] == 1].to_list()
        AD_T_500 = AD_T_500 + merge_flow_info_dict[flow].index[merge_flow_info_dict[flow]["500_T_AD"] == 1].to_list()
        AD_T_1000 = AD_T_1000 + merge_flow_info_dict[flow].index[merge_flow_info_dict[flow]["1000_T_AD"] == 1].to_list()
        AD_T_5000 = AD_T_5000 + merge_flow_info_dict[flow].index[merge_flow_info_dict[flow]["5000_T_AD"] == 1].to_list()
    AD_T_100 = sorted(list(set(AD_T_100)))
    AD_T_1000 = sorted(list(set(AD_T_1000)))
    AD_T_500 = sorted(list(set(AD_T_500)))
    AD_T_5000 = sorted(list(set(AD_T_5000)))

    AD_T_100_label = np.zeros(array_length, dtype=int)
    AD_T_100_label[AD_T_100] = 1
    AD_T_500_label = np.zeros(array_length, dtype=int)
    AD_T_500_label[AD_T_500] = 1

    AD_T_1000_label = np.zeros(array_length, dtype=int)
    AD_T_1000_label[AD_T_1000] = 1
    AD_T_5000_label = np.zeros(array_length, dtype=int)
    AD_T_5000_label[AD_T_5000] = 1

    return np.array(AD_T_100_label)[50:], np.array(AD_T_500_label)[50:], np.array(AD_T_1000_label)[50:], np.array(AD_T_5000_label)[50:]
    

In [5]:
def calc_exp_AD_efficiency_p2p(base_dir, sketch_max_ratio = 1, sketch_sigma = 3, offset = 8):
    merge_flow_info_dict = get_flows_df(base_dir)
    AD_T_100_label, AD_T_500_label, AD_T_1000_label, AD_T_5000_label = anomaly_detection_static_threshold(merge_flow_info_dict)
    threshold_per_flow = calc_threshold(merge_flow_info_dict, sketch_max_ratio, sketch_sigma)
    AD_res = anomaly_detection(merge_flow_info_dict, threshold_per_flow)
    inject_info = get_injection_info(base_dir)
    AD_res = label_AD_true_label(AD_res, inject_info, self_defined_offset= offset)
    # return AD_res
    in_network_AD_label, MARS_AD_label, real_label = calc_AD_label(AD_res)
    MARS_f1, MARS_precision, MARS_recall, MARS_TP, MARS_TN, MARS_FP, MARS_FN = calc_seq(MARS_AD_label, real_label)
    print("MARS:")
    print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(MARS_f1, MARS_precision, MARS_recall, MARS_TP, MARS_TN, MARS_FP, MARS_FN))

    SKETCH_f1, SKETCH_precision, SKETCH_recall, SKETCH_TP, SKETCH_TN, SKETCH_FP, SKETCH_FN = calc_seq(in_network_AD_label, real_label)
    print("Sketch based:")
    print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(SKETCH_f1, SKETCH_precision, SKETCH_recall, SKETCH_TP, SKETCH_TN, SKETCH_FP, SKETCH_FN))


    T_100_f1, T_100_precision, T_100_recall, T_100_TP, T_100_TN, T_100_FP, T_100_FN = calc_seq(AD_T_100_label, real_label)
    # print("T_100:")
    # print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(T_100_f1, T_100_precision, T_100_recall, T_100_TP, T_100_TN, T_100_FP, T_100_FN))

    T_1000_f1, T_1000_precision, T_1000_recall, T_1000_TP, T_1000_TN, T_1000_FP, T_1000_FN = calc_seq(AD_T_1000_label, real_label)
    # print("T_1000:")
    # print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(T_1000_f1, T_1000_precision, T_1000_recall, T_1000_TP, T_1000_TN, T_1000_FP, T_1000_FN))

    T_500_f1, T_500_precision, T_500_recall, T_500_TP, T_500_TN, T_500_FP, T_500_FN = calc_seq(AD_T_500_label, real_label)
    # print("T_500:")
    # print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(T_500_f1, T_500_precision, T_500_recall, T_500_TP, T_500_TN, T_500_FP, T_500_FN))

    T_5000_f1, T_5000_precision, T_5000_recall, T_5000_TP, T_5000_TN, T_5000_FP, T_5000_FN = calc_seq(AD_T_5000_label, real_label)
    # print("T_5000:")
    # print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(T_5000_f1, T_5000_precision, T_5000_recall, T_5000_TP, T_5000_TN, T_5000_FP, T_5000_FN))

    
    return MARS_f1, MARS_precision, MARS_recall, MARS_TP, MARS_TN, MARS_FP, MARS_FN, \
        SKETCH_f1, SKETCH_precision, SKETCH_recall, SKETCH_TP, SKETCH_TN, SKETCH_FP, SKETCH_FN,\
        T_100_f1, T_100_precision, T_100_recall, T_100_TP, T_100_TN, T_100_FP, T_100_FN,\
        T_1000_f1, T_1000_precision, T_1000_recall, T_1000_TP, T_1000_TN, T_1000_FP, T_1000_FN,\
        T_500_f1, T_500_precision, T_500_recall, T_500_TP, T_500_TN, T_500_FP, T_500_FN,\
        T_5000_f1, T_5000_precision, T_5000_recall, T_5000_TP, T_5000_TN, T_5000_FP, T_5000_FN

In [49]:
def calc_exp_AD_efficiency_range(base_dir, sketch_max_ratio = 1, sketch_sigma = 3, offset = 8):
    merge_flow_info_dict = get_flows_df(base_dir)
    AD_T_100_label, AD_T_500_label, AD_T_1000_label, AD_T_5000_label = anomaly_detection_static_threshold(merge_flow_info_dict)
    threshold_per_flow = calc_threshold(merge_flow_info_dict, sketch_max_ratio, sketch_sigma)
    AD_res = anomaly_detection(merge_flow_info_dict, threshold_per_flow)
    inject_info = get_injection_info(base_dir)
    AD_res = label_AD_true_label(AD_res, inject_info, self_defined_offset= offset)
    # return AD_res
    in_network_AD_label, MARS_AD_label, real_label = calc_AD_label(AD_res)
    adjust_in_network_AD_label = adjust_predicts(None, real_label, pred=in_network_AD_label.copy(), calc_latency=False)
    adjust_MARS_AD_label = adjust_predicts(None, real_label, pred=MARS_AD_label.copy(), calc_latency=False)
    
    # range_based_TSM(real_label, adjust_MARS_AD_label)
    # range_based_TSM(real_label, adjust_in_network_AD_label)
    df = {"real_label":real_label, "MARS_label": MARS_AD_label, "in_network_ad_label":in_network_AD_label, "adjust_in_network_label":adjust_in_network_AD_label, "adjust_mars_label":adjust_MARS_AD_label}
    return pd.DataFrame(df) 
    range_based_TSM(real_label, AD_T_100_label)
    range_based_TSM(real_label, AD_T_1000_label)

    range_based_TSM(real_label, AD_T_500_label)
    range_based_TSM(real_label, AD_T_5000_label)
    


    # in_network_AD_label, MARS_AD_label, real_label = calc_AD_label(AD_res)
    # MARS_f1, MARS_precision, MARS_recall, MARS_TP, MARS_TN, MARS_FP, MARS_FN = calc_seq(MARS_AD_label, real_label)
    # print("MARS:")
    # print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(MARS_f1, MARS_precision, MARS_recall, MARS_TP, MARS_TN, MARS_FP, MARS_FN))

    # SKETCH_f1, SKETCH_precision, SKETCH_recall, SKETCH_TP, SKETCH_TN, SKETCH_FP, SKETCH_FN = calc_seq(in_network_AD_label, real_label)
    # print("Sketch based:")
    # print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(SKETCH_f1, SKETCH_precision, SKETCH_recall, SKETCH_TP, SKETCH_TN, SKETCH_FP, SKETCH_FN))


    # T_100_f1, T_100_precision, T_100_recall, T_100_TP, T_100_TN, T_100_FP, T_100_FN = calc_seq(AD_T_100_label, real_label)
    # # print("T_100:")
    # # print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(T_100_f1, T_100_precision, T_100_recall, T_100_TP, T_100_TN, T_100_FP, T_100_FN))

    # T_1000_f1, T_1000_precision, T_1000_recall, T_1000_TP, T_1000_TN, T_1000_FP, T_1000_FN = calc_seq(AD_T_1000_label, real_label)
    # # print("T_1000:")
    # # print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(T_1000_f1, T_1000_precision, T_1000_recall, T_1000_TP, T_1000_TN, T_1000_FP, T_1000_FN))

    # T_500_f1, T_500_precision, T_500_recall, T_500_TP, T_500_TN, T_500_FP, T_500_FN = calc_seq(AD_T_500_label, real_label)
    # # print("T_500:")
    # # print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(T_500_f1, T_500_precision, T_500_recall, T_500_TP, T_500_TN, T_500_FP, T_500_FN))

    # T_5000_f1, T_5000_precision, T_5000_recall, T_5000_TP, T_5000_TN, T_5000_FP, T_5000_FN = calc_seq(AD_T_5000_label, real_label)
    # # print("T_5000:")
    # # print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(T_5000_f1, T_5000_precision, T_5000_recall, T_5000_TP, T_5000_TN, T_5000_FP, T_5000_FN))

    
    # return MARS_f1, MARS_precision, MARS_recall, MARS_TP, MARS_TN, MARS_FP, MARS_FN, \
    #     SKETCH_f1, SKETCH_precision, SKETCH_recall, SKETCH_TP, SKETCH_TN, SKETCH_FP, SKETCH_FN,\
    #     T_100_f1, T_100_precision, T_100_recall, T_100_TP, T_100_TN, T_100_FP, T_100_FN,\
    #     T_1000_f1, T_1000_precision, T_1000_recall, T_1000_TP, T_1000_TN, T_1000_FP, T_1000_FN,\
    #     T_500_f1, T_500_precision, T_500_recall, T_500_TP, T_500_TN, T_500_FP, T_500_FN,\
    #     T_5000_f1, T_5000_precision, T_5000_recall, T_5000_TP, T_5000_TN, T_5000_FP, T_5000_FN

In [52]:
import numpy as np
import matplotlib as plt
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
from statsmodels import robust

class TSMetric:
    def __init__(self, metric_option="classic", beta=1.0, alpha_r=0.0, cardinality="one", bias_p="flat", bias_r="flat"):
        
        assert (alpha_r >= 0)&(alpha_r <= 1)
        assert metric_option in ["classic", "time-series", "numenta"]
        assert beta > 0
        assert cardinality in ["one", "reciprocal", "udf_gamma"]
        assert bias_p in ["flat", "front", "middle", "back"]
        assert bias_r in ["flat", "front", "middle", "back"]
        
        self.metric_option = metric_option
        self.beta = beta
        self.alpha_r = alpha_r
        self.alpha_p = 0
        self.cardinality = cardinality
        self.bias_p = bias_p
        self.bias_r = bias_r
    
    def _udf_gamma(self, overlap, task_type):
        """
        user defined gamma
        """
        return 1.0
    
    def _gamma_select(self, gamma, overlap, task_type):
        if gamma == "one":
            return 1.0
        elif gamma == "reciprocal":
            if overlap > 1:
                return 1.0/overlap
            else:
                return 1.0
        elif gamma == "udf_gamma_def":
            if overlap > 1:
                return 1.0/self._udf_gamma(overlap, task_type)
            else:
                return 1.0
    
    def _gamma_function(self, overlap_count, task_type):
        overlap = overlap_count[0]
        if task_type == 0:
            return self._gamma_select(self.cardinality, overlap, task_type)
        elif task_type == 1:
            return self._gamma_select(self.cardinality, overlap, task_type)
        else:
            raise Exception("invalid argument in gamma function")
    
    
    def _compute_omega_reward(self, r1, r2, overlap_count, task_type):
        if r1[1] < r2[0] or r1[0] > r2[1]:
            return 0
        else:
            overlap_count[0] += 1
            overlap = np.zeros(r1.shape)
            overlap[0] = max(r1[0], r2[0])
            overlap[1] = min(r1[1], r2[1])
            return self._omega_function(r1, overlap, task_type)
    
    def _omega_function(self, rrange, overlap, task_type):
        anomaly_length = rrange[1] - rrange[0] + 1
        my_positional_bias = 0
        max_positional_bias = 0
        temp_bias = 0
        for i in range(1, anomaly_length+1):
            temp_bias = self._delta_function(i, anomaly_length, task_type)
            max_positional_bias += temp_bias
            j = rrange[0] + i -1
            if j >= overlap[0] and j <= overlap[1]:
                my_positional_bias += temp_bias
        if max_positional_bias > 0:
            res = my_positional_bias / max_positional_bias
            return res
        else:
            return 0
    
    def _delta_function(self, t, anomaly_length, task_type):
        if task_type == 0:
            return self._delta_select(self.bias_p, t, anomaly_length, task_type)
        elif task_type == 1:
            return self._delta_select(self.bias_r, t, anomaly_length, task_type)
        else:
            raise Exception("Invalid task type in delta function")
    
    def _delta_select(self, delta, t, anomaly_length, task_type):
        if delta == "flat":
            return 1.0
        elif delta == "front":
            return float(anomaly_length - t + 1.0)
        elif delta == "middle":
            if t <= anomaly_length/2.0 :
                return float(t)
            else:
                return float(anomaly_length - t + 1.0)
        elif delta == "back":
            return float(t)
        elif delta == "udf_delta":
            return self._udf_delta(t, anomaly_length, task_type)
        else:
            raise Exception("Invalid positional bias value")
    
    def _udf_delta(self, t, anomaly_length, task_type):
        """
        user defined delta function
        """
        return 1.0
    
    def _update_precision(self, real_anomalies, predicted_anomalies):
        precision = 0
        if len(predicted_anomalies) == 0:
            return 0
        for i in range(len(predicted_anomalies)):
            range_p = predicted_anomalies[i, :]
            # print(range_p)
            omega_reward = 0
            overlap_count = [0]
            for j in range(len(real_anomalies)):
                range_r = real_anomalies[j, :]
                omega_reward += self._compute_omega_reward(range_p, range_r, overlap_count, 0)
            
            overlap_reward = self._gamma_function(overlap_count, 0)*omega_reward
            print(omega_reward, overlap_count, overlap_reward)
            if overlap_count[0] > 0:
                existence_reward = 1
            else:
                existence_reward = 0

            precision += self.alpha_p*existence_reward + (1 - self.alpha_p)*overlap_reward
        precision /= len(predicted_anomalies)
        return precision
    
    def _update_recall(self, real_anomalies, predicted_anomalies):
        recall = 0
        if len(real_anomalies) == 0:
            return 0
        for i in range(len(real_anomalies)):
            omega_reward = 0
            overlap_count = [0]
            range_r = real_anomalies[i, :]
            for j in range(len(predicted_anomalies)):
                range_p = predicted_anomalies[j, :]
                omega_reward += self._compute_omega_reward(range_r, range_p, overlap_count, 1)
            overlap_reward = self._gamma_function(overlap_count, 1)*omega_reward
            
            if overlap_count[0] > 0:
                existence_reward = 1
            else:
                existence_reward = 0

            recall += self.alpha_r*existence_reward + (1 - self.alpha_r)*overlap_reward
        recall /= len(real_anomalies)
        return recall
    
    def _shift(self, arr, num, fill_value=np.nan):
        arr = np.roll(arr,num)
        if num < 0:
            arr[num:] = fill_value
        elif num > 0:
            arr[:num] = fill_value
        return arr
    
    def _prepare_data(self, values_real, values_pred):
        
        assert len(values_real) == len(values_pred)
        assert np.allclose(np.unique(values_real), np.array([0, 1]))
        assert np.allclose(np.unique(values_pred), np.array([0, 1]))
        
        if self.metric_option == "classic":
            real_anomalies = np.argwhere(values_real == 1).repeat(2, axis=1)
            predicted_anomalies = np.argwhere(values_pred == 1).repeat(2, axis=1)
            
        elif self.metric_option == "time-series":
            predicted_anomalies_ = np.argwhere(values_pred == 1).ravel()
            # print(predicted_anomalies_)
            predicted_anomalies_shift_forward = self._shift(predicted_anomalies_, 1, fill_value=predicted_anomalies_[0])
            # print(predicted_anomalies_shift_forward)
            predicted_anomalies_shift_backward = self._shift(predicted_anomalies_, -1, fill_value=predicted_anomalies_[-1])
            # print(predicted_anomalies_shift_backward)
            predicted_anomalies_start = np.argwhere((predicted_anomalies_shift_forward - predicted_anomalies_) != -1).ravel()
            predicted_anomalies_finish = np.argwhere((predicted_anomalies_ - predicted_anomalies_shift_backward) != -1).ravel()

            # print(predicted_anomalies_start, predicted_anomalies_finish)
            predicted_anomalies = np.hstack([predicted_anomalies_[predicted_anomalies_start].reshape(-1, 1), \
                                             predicted_anomalies_[predicted_anomalies_finish].reshape(-1, 1)])
            print(predicted_anomalies)
            real_anomalies_ = np.argwhere(values_real == 1).ravel()
            real_anomalies_shift_forward = self._shift(real_anomalies_, 1, fill_value=real_anomalies_[0])
            real_anomalies_shift_backward = self._shift(real_anomalies_, -1, fill_value=real_anomalies_[-1])
            real_anomalies_start = np.argwhere((real_anomalies_shift_forward - real_anomalies_) != -1).ravel()
            real_anomalies_finish = np.argwhere((real_anomalies_ - real_anomalies_shift_backward) != -1).ravel()
            real_anomalies = np.hstack([real_anomalies_[real_anomalies_start].reshape(-1, 1), \
                                             real_anomalies_[real_anomalies_finish].reshape(-1, 1)])
            # print(real_anomalies)
            
        elif self.metric_option == "numenta":
            predicted_anomalies = np.argwhere(values_pred == 1).repeat(2, axis=1)
            real_anomalies_ = np.argwhere(values_real == 1).ravel()
            real_anomalies_shift_forward = self._shift(real_anomalies_, 1, fill_value=real_anomalies_[0])
            real_anomalies_shift_backward = self._shift(real_anomalies_, -1, fill_value=real_anomalies_[-1])
            real_anomalies_start = np.argwhere((real_anomalies_shift_forward - real_anomalies_) != -1).ravel()
            real_anomalies_finish = np.argwhere((real_anomalies_ - real_anomalies_shift_backward) != -1).ravel()
            real_anomalies = np.hstack([real_anomalies_[real_anomalies_start].reshape(-1, 1), \
                                             real_anomalies_[real_anomalies_finish].reshape(-1, 1)])
        return real_anomalies, predicted_anomalies
    
    def score(self, values_real, values_predicted):
        assert isinstance(values_real, np.ndarray)
        assert isinstance(values_predicted, np.ndarray)
        real_anomalies, predicted_anomalies = self._prepare_data(values_real, values_predicted)
        # print(real_anomalies, predicted_anomalies)
        precision = self._update_precision(real_anomalies, predicted_anomalies)
        recall = self._update_recall(real_anomalies, predicted_anomalies)
        if precision + recall != 0:
            Fbeta = (1 + self.beta**2)*precision*recall/(self.beta**2*precision + recall)
        else:
            Fbeta = 0
        
        return precision, recall, Fbeta
        
        

def range_based_TSM(true_label,pred_label):
    # flat_metric = TSMetric(metric_option="time-series", alpha_r=0.5, cardinality="reciprocal", bias_p="flat", bias_r="flat")
    # precision_flat, recall_flat, f1_flat = flat_metric.score(true_label, pred_label)
    # print("flat metric")
    # print("precision: ", precision_flat, "recall: ", recall_flat, "f1: ", f1_flat)

    front_metric = TSMetric(metric_option="time-series", alpha_r=0.5, cardinality="reciprocal", bias_p="front", bias_r="front")
    precision_front, recall_front, f1_front = front_metric.score(true_label, pred_label)
    print("front metric")
    print("precision: ", precision_front, "recall: ", recall_front, "f1: ", f1_front)

    # middle_metric = TSMetric(metric_option="time-series", alpha_r=0.5, cardinality="reciprocal", bias_p="flat", bias_r="middle")
    # precision_middle, recall_middle, f1_middle = middle_metric.score(true_label, pred_label)
    # print("middle metric")
    # print("precision: ", precision_middle, "recall: ", recall_middle, "f1: ", f1_middle)

    # back_metric = TSMetric(metric_option="time-series", alpha_r=0.5, cardinality="reciprocal", bias_p="flat", bias_r="back")
    # precision_back, recall_back, f1_back = back_metric.score(true_label, pred_label)
    # print("back metric")
    # print("precision: ", precision_back, "recall: ", recall_back, "f1: ", f1_back)

    print("----------------------------------------------------------------------------------------")

In [53]:
base_dir = "./log/"
AD_res = calc_exp_AD_efficiency_range(base_dir, offset = 8)
range_based_TSM(AD_res["real_label"].values, AD_res["in_network_ad_label"].values)
range_based_TSM(AD_res["real_label"].values, AD_res["MARS_label"].values)

# range_based_TSM(AD_res["real_label"],  adjust_MARS_AD_label)
# range_based_TSM(real_label, adjust_in_network_AD_label)

[[ 14  14]
 [ 32  32]
 [ 45  46]
 [ 67  67]
 [ 89  89]
 [103 103]
 [126 126]
 [143 143]
 [178 178]
 [367 367]
 [420 420]
 [422 422]
 [436 439]]
0 [0] 0.0
0 [0] 0.0
0 [0] 0.0
0 [0] 0.0
0 [0] 0.0
0 [0] 0.0
1.0 [1] 1.0
1.0 [1] 1.0
1.0 [1] 1.0
1.0 [1] 1.0
0 [0] 0.0
0 [0] 0.0
0 [0] 0.0
front metric
precision:  0.3076923076923077 recall:  0.5024281112039136 f1:  0.381655026536551
----------------------------------------------------------------------------------------
[[  0 283]
 [285 286]
 [288 290]
 [292 294]
 [309 309]
 [311 311]
 [313 314]
 [324 324]
 [335 335]
 [340 342]
 [351 351]
 [357 357]
 [360 360]
 [367 368]
 [372 372]
 [375 375]
 [377 377]
 [386 387]
 [393 393]
 [397 397]
 [400 400]
 [404 404]
 [419 424]
 [434 442]
 [449 449]
 [451 451]
 [454 454]
 [459 459]
 [473 473]
 [481 481]
 [483 483]
 [496 498]
 [508 508]]
0.34250061774153695 [1] 0.34250061774153695
1.0 [1] 1.0
1.0 [1] 1.0
1.0 [1] 1.0
1.0 [1] 1.0
1.0 [1] 1.0
1.0 [1] 1.0
1.0 [1] 1.0
1.0 [1] 1.0
1.0 [1] 1.0
1.0 [1] 1.0
1.0 [1

In [17]:
AD_res

,real_label,MARS_label,In_network_ad_label
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
5,0,1,0
6,0,1,0
7,0,1,0
8,0,1,0
9,0,1,0


In [90]:
latency_pd = {}
for flow_info in AD_res.keys():
    latency_pd[flow_info] = AD_res[flow_info]["latency"]
    latency_pd["label"] = AD_res[flow_info]["true_label"]
latency_pd = pd.DataFrame(latency_pd)

In [91]:
latency_pd

,h13_h11,label,h5_h10,h2_h14,h10_h4,h9_h4,h6_h12,h14_h12,h7_h9,h8_h3,h16_h13,h4_h13,h12_h16,h3_h1,h15_h1,h11_h15,h1_h7
0,24481.0,False,26113.0,21023.0,22870.0,39827.0,16520.0,21084.0,25394.0,27881.0,11473.0,22152.0,21510.0,13893.0,23258.0,18508.0,36480.0
1,18030.0,False,27080.0,18767.0,27661.0,25594.0,24355.0,21791.0,23502.0,23983.0,13466.0,27238.0,18907.0,11055.0,30976.0,20536.0,22019.0
2,22328.0,False,23245.0,24412.0,27858.0,17135.0,20620.0,30610.0,22695.0,17871.0,11386.0,20168.0,36416.0,9049.0,22885.0,20738.0,32506.0
3,22706.0,False,30379.0,25359.0,21366.0,21087.0,26457.0,15720.0,22900.0,20865.0,8679.0,21980.0,23831.0,13697.0,15269.0,19714.0,23534.0
4,16458.0,False,18921.0,26199.0,28420.0,20311.0,20491.0,18118.0,25733.0,24106.0,18203.0,24568.0,23362.0,12773.0,19339.0,23367.0,18897.0
5,22660.0,False,29898.0,18957.0,19124.0,24010.0,25463.0,20492.0,26113.0,24678.0,12364.0,17704.0,20151.0,8763.0,26207.0,17852.0,25087.0
6,26052.0,False,31511.0,22038.0,15842.0,21497.0,20812.0,28366.0,20097.0,26379.0,11308.0,24422.0,24667.0,23752.0,24632.0,23802.0,15541.0
7,28552.0,False,24250.0,20134.0,19772.0,26793.0,22979.0,22621.0,20942.0,18864.0,11204.0,21166.0,29420.0,14113.0,24001.0,20606.0,18389.0
8,20321.0,False,18387.0,22962.0,20012.0,34184.0,16926.0,16840.0,26566.0,29365.0,17805.0,39317.0,24976.0,17571.0,17055.0,17212.0,20580.0
9,27546.0,False,23420.0,19836.0,23779.0,20878.0,20915.0,17930.0,25243.0,27193.0,18605.0,16326.0,20525.0,10760.0,20352.0,15385.0,20338.0


In [183]:
def PR_F1(TP, TN, FP, FN):
    p = TP/(TP+FP+0.00001)
    r = TP/(TP+FN+0.00001)
    f1 = 2 * p * r / (p + r + 0.00001)
    return p, r, f1
def calc_FT_AD_efficiency(log_base_dir, sketch_max_ratio = 1, sketch_sigma = 3, offset = 8):
    total_MARS_TP, total_MARS_TN, total_MARS_FP, total_MARS_FN = 0,0,0,0
    total_SKETCH_TP, total_SKETCH_TN, total_SKETCH_FP, total_SKETCH_FN = 0,0,0,0

    total_100_TP, total_1000_TP, total_500_TP, total_5000_TP = 0,0,0,0
    total_100_TN, total_1000_TN, total_500_TN, total_5000_TN = 0,0,0,0
    total_100_FP, total_1000_FP, total_500_FP, total_5000_FP = 0,0,0,0
    total_100_FN, total_1000_FN, total_500_FN, total_5000_FN = 0,0,0,0


    for exp_id in os.listdir(log_base_dir):
        base_dir = os.path.join(log_base_dir, exp_id)
        MARS_f1, MARS_precision, MARS_recall, MARS_TP, MARS_TN, MARS_FP, MARS_FN, \
            SKETCH_f1, SKETCH_precision, SKETCH_recall, SKETCH_TP, SKETCH_TN, SKETCH_FP, SKETCH_FN,\
            T_100_f1, T_100_precision, T_100_recall, T_100_TP, T_100_TN, T_100_FP, T_100_FN,\
            T_1000_f1, T_1000_precision, T_1000_recall, T_1000_TP, T_1000_TN, T_1000_FP, T_1000_FN,\
            T_500_f1, T_500_precision, T_500_recall, T_500_TP, T_500_TN, T_500_FP, T_500_FN,\
            T_5000_f1, T_5000_precision, T_5000_recall, T_5000_TP, T_5000_TN, T_5000_FP, T_5000_FN  = calc_exp_AD_efficiency(base_dir,sketch_max_ratio, sketch_sigma , offset)
        
        total_MARS_TP, total_MARS_TN, total_MARS_FP, total_MARS_FN =\
             total_MARS_TP + MARS_TP, total_MARS_TN + MARS_TN, total_MARS_FP + MARS_FP, total_MARS_FN + MARS_FN

        total_SKETCH_TP, total_SKETCH_TN, total_SKETCH_FP, total_SKETCH_FN =\
             total_SKETCH_TP + SKETCH_TP, total_SKETCH_TN + SKETCH_TN, total_SKETCH_FP + SKETCH_FP, total_SKETCH_FN + SKETCH_FN


        total_100_TP, total_100_TN, total_100_FP, total_100_FN = \
            total_100_TP + T_100_TP, total_100_TN + T_100_TN, total_100_FP + T_100_FP, total_100_FN + T_100_FN

        total_1000_TP, total_1000_TN, total_1000_FP, total_1000_FN = \
            total_1000_TP + T_1000_TP, total_1000_TN + T_1000_TN, total_1000_FP + T_1000_FP, total_1000_FN + T_1000_FN
        

        total_500_TP, total_500_TN, total_500_FP, total_500_FN = \
            total_500_TP + T_500_TP, total_500_TN + T_500_TN, total_500_FP + T_500_FP, total_500_FN + T_500_FN

        total_5000_TP, total_5000_TN, total_5000_FP, total_5000_FN = \
            total_5000_TP + T_5000_TP, total_5000_TN + T_5000_TN, total_5000_FP + T_5000_FP, total_5000_FN + T_5000_FN


    print("**************")
    MARS_precision, MARS_recall, MARS_f1 = PR_F1(total_MARS_TP, total_MARS_TN, total_MARS_FP, total_MARS_FN)
    print("MARS:")
    print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(MARS_f1, MARS_precision, MARS_recall, total_MARS_TP, total_MARS_TN, total_MARS_FP, total_MARS_FN))


    SKETCH_precision, SKETCH_recall, SKETCH_f1 = PR_F1(total_SKETCH_TP, total_SKETCH_TN, total_SKETCH_FP, total_SKETCH_FN)
    print("Sketch based:")
    print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(SKETCH_f1, SKETCH_precision, SKETCH_recall, total_SKETCH_TP, total_SKETCH_TN, total_SKETCH_FP, total_SKETCH_FN))
        
    T_100_precision, T_100_recall, T_100_f1 = PR_F1(total_100_TP, total_100_TN, total_100_FP, total_100_FN)
    print("100 Threshold:")
    print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(T_100_f1, T_100_precision, T_100_recall, total_100_TP, total_100_TN, total_100_FP, total_100_FN))

    T_1000_precision, T_1000_recall, T_1000_f1 = PR_F1(total_1000_TP, total_1000_TN, total_1000_FP, total_1000_FN)
    print("1000 Threshold:")
    print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(T_1000_f1, T_1000_precision, T_1000_recall, total_1000_TP, total_1000_TN, total_1000_FP, total_1000_FN))
        
    T_500_precision, T_500_recall, T_500_f1 = PR_F1(total_500_TP, total_500_TN, total_500_FP, total_500_FN)
    print("500 Threshold:")
    print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(T_500_f1, T_500_precision, T_500_recall, total_500_TP, total_500_TN, total_500_FP, total_500_FN))

    T_5000_precision, T_5000_recall, T_5000_f1 = PR_F1(total_5000_TP, total_5000_TN, total_5000_FP, total_5000_FN)
    print("5000 Threshold:")
    print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(T_5000_f1, T_5000_precision, T_5000_recall, total_5000_TP, total_5000_TN, total_5000_FP, total_5000_FN))
        

    return total_MARS_TP, total_MARS_TN, total_MARS_FP, total_MARS_FN,\
            total_SKETCH_TP, total_SKETCH_TN, total_SKETCH_FP, total_SKETCH_FN,\
            total_100_TP, total_100_TN, total_100_FP, total_100_FN,\
            total_1000_TP, total_1000_TN, total_1000_FP, total_1000_FN,\
            total_500_TP, total_500_TN, total_500_FP, total_500_FN,\
            total_5000_TP, total_5000_TN, total_5000_FP, total_5000_FN



In [187]:

FT_info = {"burst":[1,3,8], "port_queue":[1,3,8], "drop_bw": [2.5,1,2], "delay":[1,3,8], "ecmp_imbalance":[1,3,2]}
base_dir = "/mnt/netscope/data/netscope-sketch/"
total_MARS_TP, total_MARS_TN, total_MARS_FP, total_MARS_FN,\
            total_SKETCH_TP, total_SKETCH_TN, total_SKETCH_FP, total_SKETCH_FN,\
            total_100_TP, total_100_TN, total_100_FP, total_100_FN,\
            total_1000_TP, total_1000_TN, total_1000_FP, total_1000_FN,\
            total_500_TP, total_500_TN, total_500_FP, total_500_FN,\
            total_5000_TP, total_5000_TN, total_5000_FP, total_5000_FN = 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0
for FT,info in FT_info.items():
    print("====================="+FT+"=====================")
    log_base_dir = base_dir + FT
    FT_MARS_TP, FT_MARS_TN, FT_MARS_FP, FT_MARS_FN,\
            FT_SKETCH_TP, FT_SKETCH_TN, FT_SKETCH_FP, FT_SKETCH_FN,\
            FT_100_TP, FT_100_TN, FT_100_FP, FT_100_FN,\
            FT_1000_TP, FT_1000_TN, FT_1000_FP, FT_1000_FN,\
            FT_500_TP, FT_500_TN, FT_500_FP, FT_500_FN,\
            FT_5000_TP, FT_5000_TN, FT_5000_FP, FT_5000_FN = calc_FT_AD_efficiency(log_base_dir, sketch_max_ratio = info[0], sketch_sigma = info[1], offset = info[2])
    total_MARS_TP, total_MARS_TN, total_MARS_FP, total_MARS_FN = total_MARS_TP + FT_MARS_TP, total_MARS_TN + FT_MARS_TN, total_MARS_FP + FT_MARS_FP, total_MARS_FN + FT_MARS_FN
    total_SKETCH_TP, total_SKETCH_TN, total_SKETCH_FP, total_SKETCH_FN = total_SKETCH_TP + FT_SKETCH_TP, total_SKETCH_TN + FT_SKETCH_TN, total_SKETCH_FP + FT_SKETCH_FP, total_SKETCH_FN + FT_SKETCH_FN
    total_100_TP, total_100_TN, total_100_FP, total_100_FN = total_100_TP + FT_100_TP, total_100_TN + FT_100_TN, total_100_FP + FT_100_FP, total_100_FN + FT_100_FN
    total_1000_TP, total_1000_TN, total_1000_FP, total_1000_FN = total_1000_TP + FT_1000_TP, total_1000_TN + FT_1000_TN, total_1000_FP + FT_1000_FP, total_1000_FN + FT_1000_FN
    
    total_500_TP, total_500_TN, total_500_FP, total_500_FN = total_500_TP + FT_500_TP, total_500_TN + FT_500_TN, total_500_FP + FT_500_FP, total_500_FN + FT_500_FN
    total_5000_TP, total_5000_TN, total_5000_FP, total_5000_FN = total_5000_TP + FT_5000_TP, total_5000_TN + FT_5000_TN, total_5000_FP + FT_5000_FP, total_5000_FN + FT_5000_FN
    
print("----------Total----------")
MARS_precision, MARS_recall, MARS_f1 = PR_F1(total_MARS_TP, total_MARS_TN, total_MARS_FP, total_MARS_FN)
print("MARS:")
print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(MARS_f1, MARS_precision, MARS_recall, total_MARS_TP, total_MARS_TN, total_MARS_FP, total_MARS_FN))


SKETCH_precision, SKETCH_recall, SKETCH_f1 = PR_F1(total_SKETCH_TP, total_SKETCH_TN, total_SKETCH_FP, total_SKETCH_FN)
print("Sketch based:")
print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(SKETCH_f1, SKETCH_precision, SKETCH_recall, total_SKETCH_TP, total_SKETCH_TN, total_SKETCH_FP, total_SKETCH_FN))
        
T_100_precision, T_100_recall, T_100_f1 = PR_F1(total_100_TP, total_100_TN, total_100_FP, total_100_FN)
print("100 Threshold:")
print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(T_100_f1, T_100_precision, T_100_recall, total_100_TP, total_100_TN, total_100_FP, total_100_FN))

T_1000_precision, T_1000_recall, T_1000_f1 = PR_F1(total_1000_TP, total_1000_TN, total_1000_FP, total_1000_FN)
print("1000 Threshold:")
print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(T_1000_f1, T_1000_precision, T_1000_recall, total_1000_TP, total_1000_TN, total_1000_FP, total_1000_FN))
        
T_500_precision, T_500_recall, T_500_f1 = PR_F1(total_500_TP, total_500_TN, total_500_FP, total_500_FN)
print("500 Threshold:")
print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(T_500_f1, T_500_precision, T_500_recall, total_500_TP, total_500_TN, total_500_FP, total_500_FN))

T_5000_precision, T_5000_recall, T_5000_f1 = PR_F1(total_5000_TP, total_5000_TN, total_5000_FP, total_5000_FN)
print("5000 Threshold:")
print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(T_5000_f1, T_5000_precision, T_5000_recall, total_5000_TP, total_5000_TN, total_5000_FP, total_5000_FN))
        


=====================burst=====================
MARS:
F1:0.8281200183389521, Precision:0.706666572444457, Recall:0.9999998113207903, TP:53, TN:333, FP:22, FN:0
Sketch based:
F1:0.9380479505315988, Precision:0.8833331861111356, Recall:0.9999998113207903, TP:53, TN:348, FP:7, FN:0
MARS:
F1:0.6022684497968054, Precision:0.43089427391103463, Recall:0.9999998113207903, TP:53, TN:278, FP:70, FN:0
Sketch based:
F1:0.7571380439067208, Precision:0.6091953322763985, Recall:0.9999998113207903, TP:53, TN:314, FP:34, FN:0
MARS:
F1:0.6749954437795044, Precision:0.5094339142043477, Recall:0.999999814814849, TP:54, TN:302, FP:52, FN:0
Sketch based:
F1:0.8639949542678671, Precision:0.7605632731601023, Recall:0.999999814814849, TP:54, TN:337, FP:17, FN:0
**************
MARS:
F1:0.6896506242864002, Precision:0.5263157721606654, Recall:0.9999999375000039, TP:160, TN:913, FP:144, FN:0
Sketch based:
F1:0.8465559195152831, Precision:0.7339449204612422, Recall:0.9999999375000039, TP:160, TN:999, FP:58, FN:0
1

In [160]:
merge_flow_info_dict = merge_flow_info(raw_flow_latency_df, raw_flow_max_value_df, raw_flow_min_value_df, raw_flow_qunatile_df, raw_flow_recv_ts_df,raw_flow_MARS_AD_df)

NameError: name 'raw_flow_latency_df' is not defined

In [29]:
threshold_per_flow = calc_threshold(merge_flow_info_dict)

In [30]:
def label_AD_true_label(AD_res, inject_info, self_defined_offset=8):
    fault_type = list(inject_info.keys())[0]
    for flow in AD_res.keys():
        AD_res[flow]["true_label"] = (AD_res[flow]['recv_ts'] > inject_info[fault_type][0]["inject_t"]) & (AD_res[flow]['recv_ts'] < inject_info[fault_type][0]["inject_t"] + (inject_info[fault_type][0]["timeout"]+self_defined_offset)*1e8)
    return AD_res


In [31]:
AD_res = anomaly_detection(merge_flow_info_dict, threshold_per_flow)
inject_info = get_injection_info(base_dir)
AD_res = label_AD_true_label(AD_res, inject_info)

In [32]:
for flow in AD_res.keys():
    if 1 in AD_res[flow]["pred_label"]:
        print("Sketch_AD:", flow, AD_res[flow].iloc[15:].index[AD_res[flow]["pred_label"].iloc[15:] == 1].to_list())
        print("MARS_AD:", flow,AD_res[flow].iloc[15:].index[AD_res[flow]["MARS_AD"].iloc[15:] == 1].to_list() )

Sketch_AD: h13_h11 [201, 202, 203, 204, 205, 206, 207]
MARS_AD: h13_h11 [198, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211]
Sketch_AD: h5_h10 [419]
MARS_AD: h5_h10 [110, 137, 419, 441]
Sketch_AD: h2_h14 [387, 422]
MARS_AD: h2_h14 [123]
Sketch_AD: h10_h4 [23]
MARS_AD: h10_h4 [23, 36]
Sketch_AD: h9_h4 []
MARS_AD: h9_h4 []
Sketch_AD: h6_h12 [139, 296, 297, 298, 299, 300, 301, 302, 303, 304]
MARS_AD: h6_h12 [139, 292, 293, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323]
Sketch_AD: h14_h12 [184, 185, 186, 187, 188, 189, 236]
MARS_AD: h14_h12 [86, 182, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199]
Sketch_AD: h7_h9 [142]
MARS_AD: h7_h9 [142, 292]
Sketch_AD: h8_h3 []
MARS_AD: h8_h3 [186, 251]
Sketch_AD: h4_h13 [292, 383]
MARS_AD: h4_h13 [292]
Sketch_AD: h16_h13 [292]
MARS_AD: h16_h13 [292]
Sketch_AD: h12_h16 [81, 82, 83, 85, 86, 87, 88, 89, 90, 99]
MARS_AD: h12

In [33]:
in_network_AD_label, MARS_AD_label, real_label = calc_AD_label(AD_res)

In [56]:
# MARS_predict = adjust_predicts(score=None, label = real_label, pred = MARS_AD_label)
# in_network_AD_predict = adjust_predicts(score=None, label = real_label, pred = in_network_AD_label)

# # f1, precision, recall, TP, TN, FP, FN = calc_seq(None, label = real_label, pred = MARS_AD_label)

In [35]:
f1, precision, recall, TP, TN, FP, FN = calc_seq(MARS_AD_label, real_label)
print("MARS:")
print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(f1, precision, recall, TP, TN, FP, FN))

MARS:
F1:0.6022684497968054, Precision:0.43089427391103463, Recall:0.9999998113207903, TP:53, TN:278, FP:70, FN:0


In [36]:
f1, precision, recall, TP, TN, FP, FN = calc_seq(in_network_AD_label, real_label)
print("Sketch based:")
print("F1:{}, Precision:{}, Recall:{}, TP:{}, TN:{}, FP:{}, FN:{}".format(f1, precision, recall, TP, TN, FP, FN))

Sketch based:
F1:0.7571380439067208, Precision:0.6091953322763985, Recall:0.9999998113207903, TP:53, TN:314, FP:34, FN:0
